# 1일차 복습 (음성인식)

In [1]:
!pip install -q transformers accelerate sentencepiece gtts

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 6.8 MB/s eta 0:00:00


In [2]:
import transformers

print(transformers.__version__)

4.57.1


In [3]:
from transformers import pipeline

speech_recognizer = pipeline(
    "automatic-speech-recognition",
    model="openai/whisper-small"
)

audio_url = "https://huggingface.co/datasets/Narsil/asr_dummy/resolve/main/mlk.flac"

print("음성 인식 중")

result = speech_recognizer(audio_url)

print(result['text'])


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/967M [00:00<?, ?B/s]

generation_config.json: 0.00B [00:00, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

...

preprocessor_config.json: 0.00B [00:00, ?B/s]

Device set to use cuda:0


음성 인식 중


`return_token_timestamps` is deprecated for WhisperFeatureExtractor and will be removed in Transformers v5. Use `return_attention_mask` instead, as the number of frames can be inferred from it.
Using custom `forced_decoder_ids` from the (generation) config. This is deprecated in favor of the `task` and `language` flags/config options.
Transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English. This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`. See https://github.com/huggingface/transformers/pull/28687 for more details.


 I have a dream that one day this nation will rise up and live out the true meaning of its creed.


In [5]:
from gtts import gTTS
import os

# 생성할 텍스트(한국어)

text = """
오늘은 음성 인식과 관련한 위스퍼 모델을 복습해봅니다.
위스퍼 모델은 대중적이면서도 자막 생성 및 회의록 작성에 활용됩니다.
감사합니다. 이것으로 테스트를 마치겠습니다.
"""

# 음성 파일 생성
print("음성 파일 생성")
tts = gTTS(text=text,lang='ko',slow=False)

# 파일 저장
output_file = "example_audio.mp3"
tts.save(output_file)

print("음성 파일 생성완료")
print(f"파일 크기: {os.path.getsize(output_file)/ 1024:.2f} KB" )

audio_url = output_file

from IPython.display import Audio, display

print("음성 재생")
display(Audio(audio_url))

result_with_time = speech_recognizer(audio_url,return_timestamps=True)

print("타임스탬프를 포함합니다.")
# chunks: 텍스트를 의미있는 덩어리로 나눈 것

for chunk in result_with_time['chunks']:
  # timestamp: (시작시간, 끝시간) 튜플형태
  start,end = chunk['timestamp']
  text = chunk['text']
  print(f"[{start:.1f}s - {end:.1f}s: {text}]")

음성 파일 생성
음성 파일 생성완료
파일 크기: 124.31 KB
음성 재생


타임스탬프를 포함합니다.
[0.0s - 11.8s:  오늘은 음성 인식과 관련한 위스퍼모델을 복습해 봅니다. 위스퍼모델은 대중적이면서도 자막 생성 및 회의록 작성에 활용됩니다.]
[11.8s - 17.4s:  감사합니다. 이것으로 테스트를 마치겠습니다.]


# 2일차

## 토큰나이저 (토큰처리에 관하여)

In [6]:
!pip install -q transformers accelerate bitsandbytes sentencepiece

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 MB 13.3 MB/s eta 0:00:00


In [14]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

text = "Hello, how are you?"

# 1단계: 텍스트 → 토큰 (문자열 조각들)
tokens = tokenizer.tokenize(text)
print(f"토큰 {tokens}")

# 2단계: 토큰 → 숫자 ID
token_ids = tokenizer.convert_tokens_to_ids(tokens)
print(f"토큰 ID: {token_ids}")

# 3단계: 한번에 처리
encoded = tokenizer(text,return_tensors="pt")
print(encoded)
print(f"input_ids: {encoded['input_ids']}")
print(f"어텐션 마스크: {encoded['attention_mask']}")

print(f"\n💡 어텐션 마스크란?")
print("  1 = 실제 토큰 (AI가 집중해야 할 부분)")
print("  0 = 패딩 토큰 (AI가 무시해야 할 부분)")

### 🎪 서커스 관람 비유
'''
서커스를 보러 갔는데 10개 좌석이 있어요:
좌석: [👤 👤 👤 🪑 🪑 🪑 🪑 🪑 🪑 🪑]
마스크: [1  1  1  0  0  0  0  0  0  0]
'''

# 예시: 배치 처리
texts = ["Hello",
         "Hi, how are you doing today?",
         "Hi, how are you doing today? and what's wrong with you?",
         "오늘은 음성 인식과 관련한 위스퍼 모델을 복습해봅니다.위스퍼 모델은 대중적이면서도 자막 생성 및 회의록 작성에 활용됩니다.감사합니다. 이것으로 테스트를 마치겠습니다."]
batch_encoded = tokenizer(texts, return_tensors="pt", padding=True)

print(f"📦 배치 인코딩:")
for i, text in enumerate(texts):
    print(f"  텍스트 {i+1}: {text}")
    print(f"    IDs:  {batch_encoded['input_ids'][i].tolist()}")
    print(f"    Mask: {batch_encoded['attention_mask'][i].tolist()}")

#

토큰 ['hello', ',', 'how', 'are', 'you', '?']
토큰 ID: [7592, 1010, 2129, 2024, 2017, 1029]
{'input_ids': tensor([[ 101, 7592, 1010, 2129, 2024, 2017, 1029,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1]])}
input_ids: tensor([[ 101, 7592, 1010, 2129, 2024, 2017, 1029,  102]])
어텐션 마스크: tensor([[1, 1, 1, 1, 1, 1, 1, 1]])

💡 어텐션 마스크란?
  1 = 실제 토큰 (AI가 집중해야 할 부분)
  0 = 패딩 토큰 (AI가 무시해야 할 부분)
📦 배치 인코딩:
  텍스트 1: Hello
    IDs:  [101, 7592, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
    Mask: [1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

AI는 **한 번에 여러 문장**을 처리하려면 모든 문장이 **같은 길이**여야 해요!

In [9]:
from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer
import torch

class LightweightChatbot:

  def __init__(self):
    print("챗봇 로딩 중....")

    model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
    # AutoTokenize = 우리말 => AI가 이해하는 숫자로 변경
    self.tokenizer = AutoTokenizer.from_pretrained(model_name)
    # AutoModelForCausalLM => 다음 단어를 예측합니다.
    self.model = AutoModelForCausalLM.from_pretrained(
        model_name,
        torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
        device_map="auto"
    )
    # float32 = 4K 초고화질 (용량 큼)
    # float16 = Full HD (용량 작고 충분히 선명)

    # TextStreamer => 실시간 출력기
    self.streamer = TextStreamer(
        self.tokenizer,
        skip_prompt=True, # 사용자가 입력한 질문은 다시 출력 안 함
        skip_special_tokens=True # 특수 기호 숨기기
    )
    # 대화 내역을 저장하는 리스트
    self.messages = []

    '''
    self.messages = [
      {"role": "user", "content": "안녕?"},
      {"role": "assistant", "content": "안녕하세요!"},
      {"role": "user", "content": "날씨 어때?"},
      {"role": "assistant", "content": "오늘 날씨 좋네요!"}
    ]
    '''

    print("준비완료")

  # 실제 챗봇 기능
  def chat_once(self,user_input):
    self.messages.append({
        "role":"user",
        "content": user_input
    })

    # 채팅 템플릿 적용
    # 사용이유는 AI는 특별한 형식으로 대화를 받는다.

    text = self.tokenizer.apply_chat_template(
        self.messages,
        tokenize=False, # 일단 숫자로 변경하지 않고, 텍스트로만 만들어줘.
        add_generation_prompt=True # 마지막에 <|assistant|> 추가해서 -> 이제 AI가 답할 차례라고 표시하는 것
    )

    '''
    self.messages = [
      {"role": "user", "content": "안녕?"},
      {"role": "assistant", "content": "안녕하세요!"},
      {"role": "user", "content": "날씨 어때?"}
    ]

    ↓ apply_chat_template ↓

    변환 결과:
    "<|system|>
    You are a helpful assistant.
    <|user|>
    안녕?
    <|assistant|>
    안녕하세요!
    <|user|>
    날씨 어때?
    <|assistant|>"

    '''
    inputs = self.tokenizer([text], return_tensors="pt").to(self.model.device)
    # [text]: 리스트로 감싸기 (배치 처리 형식)
    # return_tensors="pt": PyTorch 텐서 형식으로 반환
    # .to(self.model.device) 모델이 있는 장소(GPU/CPU)로 데이터 이동

    print("🤖 Bot: ", end="", flush=True)

    # 스트리밍으로 생성 (실시간으로 출력)
    outputs = self.model.generate(
        **inputs, # 언패킹
        max_new_tokens = 200, # 결과가 최대 200개의 단어까지만 생성,
        streamer=self.streamer,
        do_sample=True, # 랜덤한 샘플링을 사용한다. 답변이 매번 조금씩 다름
        temperature=0.7, # 창의성 조절 (0.1 ~ 2.0) 숫자가 높을수록 창의적이나, 답변이 이상할 수 있다.
        top_p=0.9, # 상위 90% 확률 단어만 고려.
        repetition_penalty=1.1 # 같은말을 반복 방지 1.0 => (없음),1.1 (있음)
      )
    #  숫자 → 텍스트 변환 (생성된 답변을 추출하는 단계)
    generated = self.tokenizer.decode(outputs[0],skip_special_tokens=True)
    """
    outputs[0] = [101, 7592, 2129, ...]
            ↓ decode
    "<|system|> ... <|user|> 안녕? <|assistant|> 안녕하세요!"
    """

    # 마지막 답변만 추출
    if "<|assistant|>" in generated:
      assistant_response = generated.split("<|assistant|>")[-1].strip()
    else:
      assistant_response = generated[len(text):].strip()

    '''
    **왜 필요한가요?**

    AI는 **전체 대화를 출력**해요:

    생성 결과:
    "<|user|> 안녕? <|assistant|> 안녕하세요! <|user|> 날씨 어때? <|assistant|> 오늘 화창해요!"


    우리가 원하는 건 **마지막 답변**만:

    "오늘 화창해요!"

    '''

    self.messages.append(
        {"role": "assistant", "content": assistant_response}
    )

    # 줄바꿈
    print()

bot = LightweightChatbot() # 인스턴스 만들기

# 대화 예시
# bot.chat_once("Hi! What's the capital of France?")
# bot.chat_once("What's special about that city?")
# bot.chat_once("Tell me a fun fact!")
bot.chat_once("When did the theory of deep learning emerge?")


챗봇 로딩 중....
준비완료
🤖 Bot: The theory of deep learning was first proposed by Yann LeCun, Yoshua Bengio, and Geoffrey Hinton in a paper titled "Gradient-based Learning for Visual Recognition" published in the journal Neural Networks. The paper introduced the concept of deep neural networks, which are composed of multiple layers with increasingly complex connections between neurons. It also established the idea that training a deep neural network requires learning patterns from vast amounts of data over a long period of time, rather than just learning specific parameters or features. Deep learning has been a significant development in artificial intelligence and continues to shape modern machine learning techniques.



# FastAPI + HuggingFace + ngrok(도메인) + 배포

In [15]:
def normal_function():
    return [1, 2, 3, 4, 5]

# 제너레이터 함수 (yield 사용)
def generator_function():
    yield 1
    yield 2
    yield 3
    yield 4
    yield 5

normal = normal_function()
print("일반 함수:", normal)  # [1, 2, 3, 4, 5] - 한번에 모든 값 반환

generator = generator_function()
print("제너레이터:", generator)

print(next(generator))
print(next(generator))
print(next(generator))


def number_generator(n):
    for i in range(1, n + 1):
        yield i * i  # 제곱값 생성

# for 루프는 자동으로 next()를 호출
print("제곱수 생성:")
for num in number_generator(5):
    print(num, end=" ")  # 1 4 9 16 25


일반 함수: [1, 2, 3, 4, 5]
제너레이터: <generator object generator_function at 0x7dfa3adaf320>
1
2
3
제곱수 생성:
1 4 9 16 25 

In [1]:
# 라이브러리 설치
!pip install -q fastapi uvicorn pyngrok nest-asyncio transformers torch

In [ ]:
# 2단계: 모델 먼저 다운로드 (서버 시작 전)
print("📦 모델 사전 다운로드 중...")

from transformers import pipeline
import torch

# 모델을 미리 다운로드하고 캐시
device = 0 if torch.cuda.is_available() else -1

try:
    sentiment_analyzer = pipeline(
        "sentiment-analysis",
        model="distilbert/distilbert-base-uncased-finetuned-sst-2-english",
        device=device
    )
    print("✅ 모델 다운로드 완료!")

    # 테스트
    test_result = sentiment_analyzer("I love this!")
    print(f"테스트 결과: {test_result}")

except Exception as e:
    print(f"❌ 모델 다운로드 실패: {e}")

📦 모델 사전 다운로드 중...


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

In [7]:
# 3단계: 서버 시작 (모델이 이미 캐시되어 있음)
import nest_asyncio
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
from contextlib import asynccontextmanager
import uvicorn
from threading import Thread
import time
from pyngrok import ngrok
import requests

nest_asyncio.apply()

# 모델 저장소
models = {}

@asynccontextmanager
async def lifespan(app: FastAPI):
    """앱 생명주기 관리"""
    print("🤖 모델 로딩 중.....")
    from transformers import pipeline
    import torch

    device = 0 if torch.cuda.is_available() else -1

    # 이미 캐시된 모델을 빠르게 로드
    models["sentiment"] = pipeline(
        "sentiment-analysis",
        model="distilbert/distilbert-base-uncased-finetuned-sst-2-english",
        device=device
    )

    print("✅ 모델 로딩 완료!")
    yield

    print("🔄 서버 종료 중...")
    models.clear()

app = FastAPI(
    title="AI API Server",
    description="Huggingface 모델을 활용한 AI API 서버",
    version="0.0.1",
    lifespan=lifespan
)

# 요청 / 응답 모델
class TextRequest(BaseModel):
    text: str

    class Config:
        json_schema_extra = {
            "example": {
                "text": "I love this product!"
            }
        }

class SentimentResponse(BaseModel):
    text: str
    sentiment: str
    confidence: float

# 엔드포인트
@app.get("/")
def root():
    return {
        "message": "AI API product is running!",
        "docs": "/docs",
        "endpoints": {
            "/analyze": "감정 분석 (POST)",
            "/health": "서버 상태 (GET)"
        }
    }

@app.post("/analyze", response_model=SentimentResponse)
def analyze_sentiment(request: TextRequest):
    """텍스트 감정 분석"""
    if not request.text.strip():
        raise HTTPException(status_code=400, detail="텍스트가 비어있습니다.")

    result = models['sentiment'](request.text)[0]

    return SentimentResponse(
        text=request.text,
        sentiment=result['label'],
        confidence=result['score']
    )

@app.get("/health")
def health_check():
    """서버 상태 확인"""
    import torch

    return {
        "status": "healthy",
        "models_loaded": list(models.keys()),
        "gpu_available": torch.cuda.is_available(),
        "gpu_name": torch.cuda.get_device_name(0) if torch.cuda.is_available() else "N/A"
    }

def wait_for_server(max_retries=30, delay=2):
    """서버가 완전히 시작될 때까지 대기"""
    print("⏳ 서버 시작 대기 중...")

    for i in range(max_retries):
        try:
            response = requests.get("http://localhost:8000/health", timeout=1)
            if response.status_code == 200:
                print(f"✅ 서버 준비 완료! ({i+1}번째 시도)")
                return True
        except requests.exceptions.RequestException:
            if i < 10:  # 처음 10번만 출력
                print(f"   시도 {i+1}/{max_retries} - 서버 대기 중...")
            time.sleep(delay)

    print("❌ 서버 시작 실패!")
    return False

def run_server():
    """Colab에서 FastAPI 서버 실행"""
    print("🚀 서버 시작 중...")

    def start_server():
        uvicorn.run(app, host="0.0.0.0", port=8000, log_level="info")

    server_thread = Thread(target=start_server, daemon=True)
    server_thread.start()

    # 서버 대기
    if not wait_for_server():
        print("❌ 서버를 시작할 수 없습니다.")
        return

    # ngrok 연결
    print("🌐 ngrok 터널 생성 중...")
    try:
        ngrok.set_auth_token("34EpYlFxapxBb7WdDmn0dkB23dL_2P5SLzhqg43EL6L5dFERH")
        public_url = ngrok.connect(8000)

        print("\n" + "="*60)
        print("✅ 서버가 성공적으로 실행되었습니다!")
        print("="*60)
        print(f"🌐 공개 URL: {public_url}")
        print(f"📖 API 문서: {public_url}/docs")
        print("="*60)
        print("⚠️  서버는 백그라운드에서 실행 중입니다.")
        print("💡 중지하려면 런타임을 재시작하세요.")
        print("="*60 + "\n")

    except Exception as e:
        print(f"❌ ngrok 연결 실패: {e}")

# 서버 실행
run_server()

🚀 서버 시작 중...
⏳ 서버 시작 대기 중...
   시도 1/300 - 서버 대기 중...


INFO:     Started server process [1870]
INFO:     Waiting for application startup.


AI 모델 로딩 중.....
   시도 2/300 - 서버 대기 중...
   시도 3/300 - 서버 대기 중...
   시도 4/300 - 서버 대기 중...
   시도 5/300 - 서버 대기 중...
   시도 6/300 - 서버 대기 중...
   시도 7/300 - 서버 대기 중...
   시도 8/300 - 서버 대기 중...
   시도 9/300 - 서버 대기 중...
   시도 10/300 - 서버 대기 중...
   시도 11/300 - 서버 대기 중...
   시도 12/300 - 서버 대기 중...
   시도 13/300 - 서버 대기 중...
   시도 14/300 - 서버 대기 중...


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

   시도 15/300 - 서버 대기 중...


...

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

   시도 25/300 - 서버 대기 중...
   시도 26/300 - 서버 대기 중...
   시도 27/300 - 서버 대기 중...
   시도 28/300 - 서버 대기 중...
   시도 29/300 - 서버 대기 중...
   시도 30/300 - 서버 대기 중...
   시도 31/300 - 서버 대기 중...
   시도 32/300 - 서버 대기 중...
   시도 33/300 - 서버 대기 중...
   시도 34/300 - 서버 대기 중...
   시도 35/300 - 서버 대기 중...
   시도 36/300 - 서버 대기 중...
   시도 37/300 - 서버 대기 중...
   시도 38/300 - 서버 대기 중...
   시도 39/300 - 서버 대기 중...
   시도 40/300 - 서버 대기 중...
   시도 41/300 - 서버 대기 중...
   시도 42/300 - 서버 대기 중...
   시도 43/300 - 서버 대기 중...
   시도 44/300 - 서버 대기 중...
   시도 45/300 - 서버 대기 중...


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

   시도 46/300 - 서버 대기 중...
   시도 47/300 - 서버 대기 중...
   시도 48/300 - 서버 대기 중...
   시도 49/300 - 서버 대기 중...
   시도 50/300 - 서버 대기 중...
   시도 51/300 - 서버 대기 중...
   시도 52/300 - 서버 대기 중...
   시도 53/300 - 서버 대기 중...
   시도 54/300 - 서버 대기 중...


KeyboardInterrupt: 